In [ ]:
!pip install spotipy
!pip install pymusixmatch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymusixmatch: filename=pymusixmatch-0.3-py3-none-any.whl size=5853 sha256=3f4345b68da92c33db89ec6fbf861b6ce66d312e1bc531c2b836a970f4a16efd
  Stored in directory: /root/.cache/pip/wheels/f6/4a/3d/363c797868dd139e1fcff2470ada9b005b1019ec2051be3fb9
Successfully built pymusixmatch


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import requests
from musixmatch import Musixmatch
import json

In [ ]:
# Spotify Authentication
client_credentials_manager = SpotifyClientCredentials(client_id='your client id', client_secret='your client secret')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# define the get_lyrics_from_musixmatch function
def get_lyrics_from_musixmatch(track_name, artist_name, musixmatch_api_key):
    base_url = "https://api.musixmatch.com/ws/1.1/"
    endpoint = "matcher.lyrics.get"
    params = {
        "q_track": track_name,
        "q_artist": artist_name,
        "apikey": musixmatch_api_key,
        "format": "json"
    }
    response = requests.get(base_url + endpoint, params=params)
    data = response.json()
    # Check if the API response includes the necessary data
    if data['message']['header']['status_code'] != 200 or 'lyrics' not in data['message']['body']:
        return "Lyrics not found"
    lyrics_data = data['message']['body'].get('lyrics')
    if not lyrics_data:
        return "Lyrics not found"
    return lyrics_data.get('lyrics_body', "Lyrics not found")

# Define the process_track function
def process_track(writer, track_info, musixmatch_api_key):
    track_name = track_info["name"]
    artist_name = track_info["artists"][0]["name"]
    artist_id = track_info["artists"][0]["id"]
    artist_info = sp.artist(artist_id)
    artist_genres = artist_info.get('genres', ["Genre not available"])
    track_popularity = track_info["popularity"]
    track_features = sp.audio_features(track_info["uri"])[0]
    lyrics = get_lyrics_from_musixmatch(track_name, artist_name, musixmatch_api_key)

    # Construct the row with dynamic mood columns
    row = [
        track_name,
        artist_name,
        ", ".join(artist_genres),  # Join list of genres into a string
        track_popularity,
        track_features['acousticness'] if track_features else "N/A",
        track_features['danceability'] if track_features else "N/A",
        track_features['duration_ms'] if track_features else "N/A",
        track_features['energy'] if track_features else "N/A",
        track_features['instrumentalness'] if track_features else "N/A",
        track_features['key'] if track_features else "N/A",
        track_features['liveness'] if track_features else "N/A",
        track_features['loudness'] if track_features else "N/A",
        track_features['mode'] if track_features else "N/A",
        track_features['speechiness'] if track_features else "N/A",
        track_features['tempo'] if track_features else "N/A",
        track_features['time_signature'] if track_features else "N/A",
        track_features['valence'] if track_features else "N/A",
        lyrics]
    writer.writerow(row)
# Header including mood and raw data
header = ['Track Name', 'Artist Name', 'Artist Genre','Popularity', 'Acousticness', 'Danceability', 'Duration (ms)', 'Energy', 'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode', 'Speechiness', 'Tempo', 'Time Signature', 'Valence', 'Lyrics']

In [ ]:
# Full List of playlists and artists to consider
playlist_links = ["https://open.spotify.com/playlist/37i9dQZF1DWVNQeZtY2TDM?si=f0c2de0f24b542d7",#Hit songs in recetn 10 years, 100 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWVUmQhB7PvFH?si=39b6335a680540dd",#Hit songs in 2000s, 50 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWSBcxmKiZ0B8?si=599c102f2ab54ac4",#Jay Chou's, 100 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DXbS5FrVEi0Jy?si=7005d28e12b8404a",#Stefanie Sun, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX4sj7z7Or175?si=2a7e474f66a84c3d",#S.H.E, 45 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWU2P2Iig3qNI?si=4b2feecce3024e5f",#Cheer Chen, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWWXlVI85jXLO?si=96e5c130f4d24809",#Sodagreen, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX6wL8zjJzqlv?si=350c6b7e50ed491b",#Jolin Tsai, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX6zRkWFtwV7e?si=858c7f6c3ee3411d",#Hebe Tien, 43 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DZ06evO4rpCJq?si=f8ef0d6c0b114c75",#Desers Chang, 26 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX1X2EBPaMHbR?si=2bb1203f1b804055",#Lala Hsu, 40 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3RBGU0?si=53521648918e4c89",#Accuse Five, 43 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWUEzhP7uKViP?si=6e2a4b0675f64d36",#A-mei, 61 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DXdCFr8d8AdIG?si=40281daff198401c",#Tanya Chua, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWWF3tJMTORMq?si=e798230c9af647e9",#Power Station, 51 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX92Z0nRvikjz?si=845aa36c78764a6f",#Mayday, 61 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DZ06evO0bocGL?si=71e5ea4ab58b4a42",#Penny Tai, 50 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3NlrY2?si=364629071ad54ca8",#Rene Liu, 50 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX7zXqDWRZPD1?si=530305d9fe2e40df",#Hit songs in 1990s, 50 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX9wQi513xJU5?si=95c693562dc64b89",#Fish Leong, 61 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DXdIrfUYWoPlG?si=06d1f946f83546b6",#Crowd Lu, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX2WThMh1h6wg?si=d9444f7b4167496e",#Weibird, 40 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DWVGQ3JeznhcJ?si=bbe9aa290c814abe",#Angela Chang, 41 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DX5XJAdiJ2VIR?si=7c030ae3c7f54a0c",#Cyndi Wang, 40 songs
                  "https://open.spotify.com/playlist/37i9dQZF1DXbDNlU6Ew5d9?si=3ff630260e444d76"]#Yoga Lin, 41 songs

In [ ]:
musixmatch_api_key = 'your api key'

In [ ]:
# Create the dataset
with open('TW_lyrics.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    # Process each playlist
    for playlist_link in playlist_links:
        playlist_URI = playlist_link.split("/")[-1].split("?")[0]
        for track in sp.playlist_tracks(playlist_URI)["items"]:
            process_track(writer, track["track"], musixmatch_api_key)